# Problem Set 2: Solving Wordle

**Release Date:** 2 September 2022

**Due Date:** 23:59, 10 September 2022

## Required Files
* words.txt

**Honour Code**: Note that plagiarism will not be condoned! You may discuss with your classmates and check the internet for references, but you **MUST NOT** submit code/report that is copied directly from other sources!

## Background

Wordle is a word game where the player has six tries to guess a five-letter word. After each guess, the player gains information about  which letters of the guess are in the correct position. For example, if the actual hidden word was "AGREE" and the guess was "ABORE", the player would see the following:

![AGREE_ABORE](imgs/AGREE_ABORE.png)

The green 'A' and 'E' means that the letters 'A' and 'E' are in their correct positions; the black 'B' and 'O' means that the word contains neither more 'B's nor more 'O's; and the yellow 'R' means that there is an 'R' somewhere in the word, but not in that position.

For an example of a successful guess sequence, refer to the figure below.

![DWARF_SEQUENCE](imgs/DWARF_SEQUENCE.png)

If you would like to play the original Wordle game, you can find it [here](https://www.nytimes.com/games/wordle/index.html). However, since this website only updates the word to be guessed daily, the solution to the game will remain the same throughout the day. If you would like to try out the game with a different word each time, click [here](https://metzger.media/games/wordle-archive/?random=play) instead. 

We suggest that you give it a go and familiarise yourself with the game!

**NOTE:**
When testing, we may use a word list that differs from that in words.txt. So, **DO NOT** hard code your solution based on that! However, you can rest assured that the word list provided is the full word list used in the original game of Wordle, and hence, your solver will be able to work on the original game.


## Game Details

Before you start implementing a solver for Wordle, it will be really helpful to understand the details of the game.

### Restrictions on Guess Words
We are only allowed to guess real words. To be more specific, entered words will be checked against a word list and if the word is not inside, it will be rejected.

### Interpreting the Coloured Letters
In the case of guess words with unique letters, the meanings are quite straightforward. Green means that the word contains those letters in the same position, yellow means that the word contains those letters but in another position and black means that the word does not contain those letters.

Unfortunately, when the guess word has repeated letters, things get a little more complicated.

For instance, if a repeated letter is coloured green in one position but yellow in another, it means that the former is correctly positioned but the latter exists in another position. Here is an example:

![SOGGY_GIGAS](imgs/SOGGY_GIGAS.png)

The green 'G' in this guess word means that the actual hidden word has a 'G' in the 3rd position while the yellow 'G' means that there is at least another 'G' in either the 2nd, 4th or 5th position of the actual hidden word.

If the first 'G' was black however, it would have meant that the actual hidden word **did not have a 'G' in the 1st, 2nd, 4th and 5th position**.

In the case of a repeated letter being coloured yellow in one position and black in another, it means that the actual hidden word contains **one** such letter but it is not in either of the chosen positions. For example, suppose that we have the result shown in the figure below. Then, the fact that the last 'G' is black implies that there is only one 'G', and in this case, it must be in either the 3rd or 4th position (if 'A' was not green, the 1st position would have been also a possibility). Furthermore, there is no 'G' in the 2nd and 5th position.

![ANGEL_AGING](imgs/ANGEL_AGING.png)

A last case that we shall explicitly discuss here is as follows. Suppose that a letter is repeated thrice, and it is coloured green, yellow and black in different positions. Then, it means that there are only two instances of it in the actual word, with one instance of it correctly positioned and the other incorrectly positioned. 

More specifically, if we have the result shown in the figure below, it means that there is an 'A' in the first position, and there is only one other 'A', which must be in either the 2nd or 4th position.

![AHEAP_ABACA](imgs/AHEAP_ABACA.png)

There are many more cases, but the aforementioned cases should give you a general idea of how things work. If you still have any doubts about it, checkout the function `make_evaluate_guess` that will generate the colour sequence based on the chosen actual word and the guess word.

## Part 1: Naive Solver

In this part of the problem set, we will implement a naive solver that will use the result of each guess to narrow down the possible words. With a smaller set of possible words to choose from, we can then make another guess, iteratively narrowing down the possibilities until we are in a state that has only one possible word or until we have found the word.

To be more precise, for part 1, our solver will work like this:
1. Randomly choose a word from the list of possible words.
2. After making our guess, we will narrow down our list of possible words by excluding words that cannot possibly be the actual hidden word, based on the result which we obtained from our guess.
3. Repeat steps 1 and 2 until we are left with one possible word or we guess the actual hidden word along the way.

We have abstracted the solver into different functions to make things easier. Please refer to the empty TODO sections below for these functions.

**NOTE:** in this problem set, **we are lifting the six-guesses restriction**, so it's possible to make more than six guesses.

**IMPORTANT:** your algorithms SHOULD work for words of length $n$, where $n \in \{1, 2, 3, ...\}$, not only words of length 5.

### Task 1.1: Generating the Guess Word

Write the function `generate_random_guess_word(possible_words)` that returns a random string from the list of `possible_words`. You should use Python's `random` library.

In [2]:
import os
import random

def generate_random_guess_word(possible_words):
    '''
    Generates a random word from the list of possible words.

    Parameters
    ----------
    possible_words:
        A list of strings of all possible remaining words in the word list.
    
    Returns
    -------
        A random string from possible_words.
    '''
  
    return possible_words[random.randint(0, len(possible_words) - 1)]

### Task 1.2: Filtering the Possible Word List After Guessing

Write the function `filter_available_words(guess_word, colours, possible_words)` that returns a new list of possible words after filtering out words from `possible_words` that are no longer possible, based on the latest `guess_word` and the resulting `colours` received.

In this case, `colours` is represented by a five-character-long string. For example, if its value is `"YGBBY"`, it means that the first, second, third, fourth and fifth letters in the given `guess_word` have been coloured yellow, green, black, black and yellow, respectively.

**NOTE:** if you miss out on filtering a few words that cannot possibly be the actual word, you will only be penalised slightly; however, if you filter out words that may be the actual word, you will be penalised harshly. (After all, a solver that filters out the correct word cannot be correct, can it?)

In [7]:
from turtle import pos


def filter_available_words(guess_word, colours, possible_words):
    '''
    Filters the list of possible words after making a guess.

    Parameters
    ----------
    guess_word:
        A string of the guess word.
    colours:
        A string representation of the colours of the result of the guess word.
    possible_words:
        A list of strings of all possible remaining words in the word list before this guess.
    
    Returns
    -------
        A list of possible words after making this guess.
    '''
    guess_word = guess_word.lower()

    green = "G"
    yellow = "Y"
    black = "B"
    if black not in colours and yellow not in colours:
        # Guess is the correct word
        return [guess_word]
    
    # Key: letter
    # Value: Array containing the index where the letter occured, and the number of non-black occurences
    occurence_dict = {}
    for index in range(len(guess_word)):
        letter = guess_word[index]
        if letter not in occurence_dict.keys():
            occurence_dict[letter] = [[], 0]
        occurence_dict[letter][0].append(index)
        if colours[index] != black:
            occurence_dict[letter][1] += 1
    # print(occurence_dict)

    for index in range(len(guess_word)):
        current_colour = colours[index]
        current_letter = guess_word[index]

        if current_colour == green:
            green_filter = lambda word : word[index] == current_letter
            possible_words = list(filter(green_filter, possible_words))
            # print(f"current length: {len(possible_words)}")
            # print(possible_words)

        elif current_colour == yellow:
            def yellow_filter(word):
                # Check if current letter is either not in word OR in the same index as guess word
                if current_letter not in word or word[index] == current_letter:
                    return False
                letter_occurence = occurence_dict[current_letter]
                min_num_of_letters = letter_occurence[1]
                if min_num_of_letters == 1:
                    return True
                # Reaching here means that the curr letter IS in word, NOT in curr index, and its repeated
                if word.count(current_letter) < min_num_of_letters:
                    return False
                return True
            possible_words = list(filter(yellow_filter, possible_words))
            # print(f"current length: {len(possible_words)}")
            # print(possible_words)

        elif current_colour == black:
            def black_filter(word):
                # Check if current letter is not in word
                if current_letter not in word:
                    return True
                if current_letter == word[index]:
                    return False
                # Curr letter is in this word
                letter_occurence = occurence_dict[current_letter]
                max_num_of_letters = letter_occurence[1]
                if max_num_of_letters == 0:
                    # Letter is not repeated or it is repeated but does not belong in word
                    return False
                # Reaching here means that the curr letter is repeated and IS in CORRECT word and should not be filtered out
                if word.count(current_letter) > max_num_of_letters:
                    return False
                return True
            possible_words = list(filter(black_filter, possible_words))
            # print(f"current length: {len(possible_words)}")
            # print(possible_words)
    
    return possible_words


    

### Task 1.3: Putting Them Together

Write the function `solver(word_list, evaluate_guess_func)` that returns a string of the correct answer, and `"NOT POSSIBLE"` if the solver is unable to find it (should not happen as long as the word exists in the word list).

Here, `word_list` is the initial list of all possible words, and `evaluate_guess_func` is a function that runs the Wordle-game logic, i.e. it takes in a guess as input and returns the colour sequence as a string. For example, for a game where the actual hidden word is `"tests"`, `evaluate_guess_func("wests")` will return `"BGGGG"`. 

In this problem set, you **DO NOT** have to implement `evaluate_guess_func` from scratch. Instead, you can create an instance of it with `make_evaluate_guess`, as provided below. Refer to the example test cases given to better understand how to create an instance of `evaluate_guess_func`, and use it with `solver`.

Note that `solver(word_list, evaluate_guess_func)` should be based on the algorithm described earlier.


In [4]:
# DO NOT MODIFY:
def make_evaluate_guess(word, word_list):
    def evaluate_guess(guess_word):
        if guess_word not in word_list:
            raise Exception("Guess word not in word list")
        word_length = len(word)
        if len(guess_word) != word_length:
            raise Exception("Guess word not of correct length")

        result = ['B']*word_length
        word_l = list(word)
        ignore_index = []
        for i in range(word_length):
            if i in ignore_index:
                continue
            if guess_word[i] == word_l[i]:
                result[i] = 'G'
                word_l[i] = '-'
                ignore_index.append(i)
        for i in range(word_length):
            if i in ignore_index:
                continue
            for j in range(word_length):
                if guess_word[i] == word_l[j]:
                    result[i] = 'Y'
                    word_l[j] = '-'
                    break
        return ''.join(result)
    return evaluate_guess

In [12]:
from asyncio import events


def solver(word_list, evaluate_guess_func):
    '''
    Solves the wordle game, getting the hidden word.

    Parameters
    ----------
    word_list:
        A list of strings of all words in the word list.
    evaluate_guess_func:
        A function that represents the wordle game with a hidden word.

        Parameters
        ----------
        guess_word:
            A string of the word to guess.

        Returns
        -------
            A string representation of the colours of the result of the guess word.
    
    Returns
    -------
        A string of the hidden word of evaluate_guess_func.
    '''
  
    # TODO: add your solution here and remove `raise NotImplementedError`
    while len(word_list) > 1:
        guess = word_list[random.randint(0, len(word_list) - 1)]
        colours = evaluate_guess_func(guess)
        word_list = filter_available_words(guess, colours, word_list)
    
    if len(word_list) == 0:
        return "NOT POSSIBLE"
    
    # if len(word_list) > 1:
    #     return "Whats going on"

    return word_list[0]

In [13]:
# Sample Tests

# Load word list from words.txt 
# Put the words.txt in the same directory as ps2.ipynb
dir_path = os.getcwd()
word_list = open(os.path.join(dir_path, 'words.txt'), 'r').read().splitlines()

''' Test cases for `solver` '''
tests_game = make_evaluate_guess("tests", word_list)
print(solver(word_list, tests_game)) # Should print 'tests'

cases_game = make_evaluate_guess("cases", word_list)
print(solver(word_list, cases_game)) # Should print 'cases'

# NOTE: you may want to write more (elaborate) test cases here

tests
cases


## Part 2: Smart(er?) Solver

One very important concept in AI that we discussed in class is the idea of *Exploration* versus *Exploitation*. 

You may have realised that although the solver is able to guarantee that we can find the solution in a finite number of steps, it is not very effective -- it is unlikely (except if you are very lucky) that we will be able to guess the word within six guesses (like in the real game). This is because the naive solver is only focused on *exploiting* what we have learnt. 

Here is an example that illustrates this. Let's say that we have narrowed down our search space to only P-INT. There are three possible words: 'POINT', 'PAINT' and 'PRINT'. With our naive solver, in the worst case, we will guess all three before we get the solution. However, if we were to guess a very different word like 'AROMA', we would have been able to tell which letter, out of 'O', 'A' and 'R', existed in the actual word. With this information, we would have only needed one more trial to enter the actual word. As such, with this approach, we would have only needed two attempts in total, instead of three in the worst case to get the actual word.

The aforementioned example illustrates how exploration -- by guessing the word 'AROMA', which deviates significantly from what we were working towards, namely a word in the form P-INT -- can be more beneficial than (immediate) exploitation.

With that said, the benefits of exploration is not always guaranteed. For example, if, on the extreme, we had just randomly selected a word like 'SIEVE' from the list of all words, it wouldn't have been very useful. 

This is where the exploitation vs exploration trade-off comes into play. We have to find a middle ground between making a safe play (by guessing only possible words) and guessing words that may be impossible but may give more information. If successful, we will be able to reduce the number of trials needed to find the word. In other words, it is helpful to have a heuristic or evaluation function  that makes an impossible guess if and only if it is likely that the guess will give us more information.

Now, in task 2, it is time for you to show us how deft you are at balancing the exploitation vs exploration trade-off to solve Wordle more 'smartly'!

### Task 2.1: Making Better Guesses

Implement the function `generate_smart_guess(word_list, possible_words, nth_guess)` that returns a "smarter" guess.

Remember, the ultimate goal of Wordle is to guess the word within a given number of tries (e.g. in the actual game, 6 tries when there are 5 letters). Hence, **a "smarter" guess is one that enables our solver to solve the games, on average, with fewer guesses**.

In this function, `possible_words` is the list of possible remaining words and `word_list` is the entire initial list of words. Additionally, `nth_guess` is a positive integer that indicates which trial -- since the start of the game -- this guess is for, i.e. `generate_smart_guess(word_list, possible_words, 1)` will generate the first guess, `generate_smart_guess(word_list, possible_words, 2)` will generate the second guess and so on.

Our smart(er) solver will then run the same algorithm as in part 1, but with the first step modified. In particular, it should work as follows:
1. Make an intelligent guess using `generate_smart_guess`.
2. After making our guess, we will narrow down our list of possible words by excluding words that cannot possibly be the actual hidden word, based on the result which we obtained from our guess.
3. Repeat steps 1 and 2 until we are left with one possible word or we guess the actual hidden word along the way.

You may want to code out the smart(er) solver, but it is **NOT** required. Instead, for task 2.1, you are only required to submit your implementation for `generate_smart_guess`.

**Hint:** Formulate the problem as a search problem, and think about which strategies are the most appropriate for this problem. Will local search work? Why or why not?

**Note:** The test cases provided on Coursemology only checks for the correctness of your solution, i.e. whether `generate_smart_guess` retains the actual hidden word. Therefore, passing the test cases **DOES NOT** mean that your heuristic is 'good' (and consequently, will be awarded full credit). In addition, please ensure that your implementation runs within reasonable time limits; solutions that take too long to run will be penalised.

In [1]:
def generate_smart_guess(word_list, possible_words, nth_guess):
    '''
    Generates a good guess word from the list of possible words.

    Parameters
    ----------
    word_list:
        A list of strings of all words in the word list.
    possible_words:
        A list of strings of all possible remaining words in the word list.
    nth_guess:
        A number indicating how many guesses have been made so far inclusive of this one.
    
    Returns
    -------
        A string of a good guess.
    '''
    def heuristic(guess_word, potential_answer):
        colours = make_evaluate_guess(potential_answer, word_list)(guess_word)
        result = filter_available_words(guess_word, colours, possible_words)
        
        return
  
    if nth_guess == 1:
        return "salet"
    


### Task 2.2: How does it work?

Describe what you did in Task 2.1 and explain the intuitions behind why your new approach works better than the naive filtering approach in Part 1.

In addition, discuss how you arrived at your final solution (e.g. What did you try initially? Did it work and why? What changes did you make?).

## Submission

Once you are done, please submit your work to Coursemology, by copying the right snippets of code into the corresponding box that says "Your answer," and click "Save."  After you save, you can make changes to your submission.

Once you are satisfied with what you have uploaded, click "Finalize submission."  **Note that once your submission is finalized, it is considered to be submitted for grading and cannot be changed.** If you need to undo this action, you will have to email your assigned tutor for help. Please do not finalize your submission until you are sure that you want to submit your solutions for grading. 
